In [ ]:
# Train network FINAL VERSION!!!!!
import h5py
import cv2
import numpy as np

#import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.utils.data as Data
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import datetime
import random


# input mudule file
import Py_autopilot2_module

from PIL import Image
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import imageio
import scipy.io as sio 
import pandas as pd
from torchvision.io import read_image


from sklearn.utils import shuffle   
from sklearn.model_selection import train_test_split    #for split
import math    #for pi
#visdom动态显示loss 和epoch
from visdom import Visdom
import time

#可视化工具visdom使用
#python -m visdom.server

# 实例化一个窗口
wind = Visdom()
# 初始化窗口信息
wind.line([0.], # Y的第一个点的坐标
		  [0.], # X的第一个点的坐标
		  win = 'train_loss', # 窗口的名称
		  opts = dict(title = 'train_loss') # 图像的标例
)


#超参设置
BATCH_SIZE = 128
EPOCH_NUM = 5000


torch.manual_seed(0)  #固定每次的初始化值

#load data shuffle and spilit
features, labels = Py_autopilot2_module.loadFromPickle()   #shape:(45406, 100, 100，3) (45406,)

features = np.array(features).astype(np.uint8)   #把features的数据类型转化成uint8，作为tranform.ToTensor的输入

train_features, test_features, train_label, test_label = train_test_split(features, labels, test_size=0.3,random_state=0,shuffle=False)
train_features_for_test = train_features    # for test dataset_train 31784

#图片数量没有变，仅仅dataset和label数组的长度增加一倍
#训练数据集左右翻转增加一倍数量，label增加一倍并取相反数， 原shape:(31784, 100, 100, 3) 
train_features_lr = train_features[:,:,::-1,:] #所有照片水平左右翻转
train_features = c = np.concatenate((train_features,train_features_lr),axis=0)    #针对0轴进行拼接 变成(63568, 100, 100, 3) 
train_label_lr = np.negative(train_label)
train_label = np.concatenate((train_label,train_label_lr),axis=0)          #针对0轴进行拼接 变成(63568,) 

# for Train dataset and Target dataset (Not used)
transform_train = transforms.Compose([            #尽量输入PIL image
    # transforms.Resize([h, w]),
    # transforms.RandomCrop(32, padding=4),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #输入数据必须是unint！可以是pil或者数组。将读取到的图片的维度进行转换（W,H,C转换为C,W,H）并将每个像素值除以255,转化成0-1之间。
    transforms.Normalize((0.4343687,0.492617,0.51987326), (0.22204618,0.24065882,0.29102236))   #x=(x-mean)/std， 转化成正态分布
])

# 构建MyDataset实例
dataset_train = Py_autopilot2_module.Custom_Dataset(
                                train_features,
                                train_label,
                                transform = transform_train
)

# 构建DataLoder
train_dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

model = Py_autopilot2_module.AutoPilot2Net()
model.train() #指定train的网络

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optomizerAdam = torch.optim.Adam(model.parameters(), lr=0.0001)  #0.02

for epoch in range(EPOCH_NUM):

    loss_rec = []
    loss_result = []
    for i, data in enumerate(train_dataloader):

        # 训练数据
        img_get, steer_get = data
                 
        img_get, steer_get = img_get.to(device), steer_get.to(device)  #steer_get.shape = torch.Size([batchsize])

        # 权重参数清零
        optomizerAdam.zero_grad()

        net_out = model.forward(img_get)  
        net_out = net_out.squeeze()

        loss = model.steering_acc_loss(net_out,  steer_get.float())  #double -> float , tensor
        loss.backward()#  反向传播
        optomizerAdam.step()

        loss_result = loss.detach().cpu().numpy() # tensor强制转换成numpy
        loss_rec.append(loss_result)

    train_loss_mean = np.mean(loss_rec)

    # visdom动态显示loss和epoch更新数据
    wind.line([train_loss_mean], [epoch], win = 'train_loss', update = 'append')
    time.sleep(0.5)

    # 保存整个网络
    if epoch % 100 == 99:
        if loss_result < 0.1 :
            model_name = 'models/model_{}.pkl'.format(epoch + 1)  #save each epoch
            torch.save(model, model_name)  #保存网络和参数  




In [ ]:
# Test just for BACKUP

# for test dataset_test=====================================================================
# 实例化一个窗口
wind = Visdom()
# 初始化窗口信息
wind.line([0.], # Y的第一个点的坐标
		  [0.], # X的第一个点的坐标
		  win = 'accr for dataset_test', # 窗口的名称
		  opts = dict(title = 'accr for dataset_test',legend=['loss', 'accr']) # 图像的标例
)

wind.line([0.], # Y的第一个点的坐标
		  [0.], # X的第一个点的坐标
		  win = 'steer for test_set', # 窗口的名称
		  opts = dict(title = 'steer for test_set',legend=['GD', 'Pred']) # 图像的标例
)

BATCH_SIZE_TEST =1
# test_features = np.swapaxes(test_features,1,3)    # shape:(31784, 100, 100, 3) -->(31784, 3,100, 100)   transforms.Tosensor已经做了
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.43567735,0.49298695,0.5192303), (0.22272868,0.24110594,0.29045662))     
])
dataset_test = Py_autopilot2_module.Custom_Dataset(
                                test_features,
                                test_label,
                                transform = transform_test
)
test_dataloader = DataLoader(dataset_test, batch_size=BATCH_SIZE_TEST, shuffle=False, num_workers=0)

#load network
test_net = torch.load('models/model_10000.pkl')   #load network
test_net.eval() #指定test的网络


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
test_net = test_net.to(device)


test_loss_rec = []
test_accr=[]
net_out_rec=[]
steer_get_rec =[]

for i, data in enumerate(test_dataloader):

    if i > 1000: break

    # 训练数据
    img_get, steer_get = data

    #显示输入图像，选择第0张图片
    # print first img in BATCH , plt.imshow input data should be WxHxC
    # img_sp = img_get[0,:,:,:].numpy()  
    # img_tp = np.swapaxes(img_sp,0,2)            #3x100x100 --> 100x100x3
    # temp = (img_tp - np.min(img_tp.flatten()))/(np.max(img_tp.flatten()) - np.min(img_tp.flatten()))
    # plt.imshow(temp)
    # plt.show()

    img = None
    for f in files:
        im=pl.imread(f)
        if img is None:
            img = pl.imshow(im)
        else:
            img.set_data(im)
        pl.pause(.1)
        pl.draw()
            
    img_get, steer_get = img_get.to(device), steer_get.to(device)  #steer_get.shape = torch.Size([batchsize])

    net_out = test_net.forward(img_get)  
    net_out = net_out.squeeze()
    steer_get = steer_get.squeeze()

    net_out_rec.append(net_out)
    steer_get_rec.append(steer_get)

    loss = test_net.steering_acc_loss(net_out,  steer_get.float())  #double -> float , tensor

    loss_result = loss.detach().cpu().numpy() 
    test_loss_rec.append(loss_result) # tensor强制转换成numpy

    for j in range(BATCH_SIZE_TEST):
        #精度=|预测-真值|/pi   
        accr_temp= np.abs(steer_get.detach().cpu().numpy() - net_out.detach().cpu().numpy()) / math.pi
        
        test_accr.append(accr_temp)

    # visdom动态显示loss和epoch更新数据
    wind.line([[loss_result, accr_temp]], [i], win = 'accr for dataset_test', update = 'append')
    wind.line([[steer_get.detach().cpu().numpy(), net_out.detach().cpu().numpy() ]], [i], win = 'steer for test_set', update = 'append')
    
    time.sleep(0.5)

test_loss_mean = np.mean(test_loss_rec)
print("test loss mean =", test_loss_mean)
test_accr_mean = np.mean(test_accr)
print("test accr mean =", test_accr_mean)